# Quantum Simulation of a free Schrödinger Wave
This implementation is based upon the paper **"Simulating Quantum Mechanics on a Quantum Computer"** attached in /research

In [76]:
from qiskit import QuantumCircuit
import numpy as np
from qiskit_aer import QasmSimulator

## 1. State Preparation
Using a recursive controlled rotation algorithm, we match the amplitudes of each qubit state with the given discretized values of the wave equation

###  1.1 Mock 3 values Wave amplitudes

In [77]:
psi = np.array([0.5, 0.7, 0.5], dtype=complex) # complex wavefunction
psi /= np.linalg.norm(psi)  # Normalize the wavefunction
print(psi)

[0.00111111+0.j 0.99999877+0.j 0.00111111+0.j]


### 1.2 Prepare Quantum Circuit with 3 qubits and 3 classical bits as registers
As discussed in the paper, each qubit represents a discrete position.

So n positions require n qubits, not the most optimal solution.

In [78]:
qc = QuantumCircuit(3, 3)

### 1.3 Rotating into correct state vector

#### 1.3.1 - Probability Tree

```plaintext
Amplitude Tree for |ψ> = c1|100> + c2|010> + c3|001>

Start: |000>  (amplitude = 1)
│
├─ R_y(θ1) on q0
│   ├─ |100> :  sin(θ1/2) = c1
│   └─ |000> :  cos(θ1/2) = √(1 - c1²)
│
├─ Controlled R_y(θ2) on q1  (control q0 = 0)
│   ├─ |010> :  √(1 - c1²) * sin(θ2/2) = c2
│   └─ |000> :  √(1 - c1²) * cos(θ2/2) = √(1 - c1² - c2²)
│
└─ Controlled R_y(θ3) on q2  (controls q0 = 0, q1 = 0)
    ├─ |001> :  √(1 - c1² - c2²) * sin(θ3/2) = c3
    └─ |000> :  √(1 - c1² - c2²) * cos(θ3/2) = 0
```

Start: |000>

1. Rotate qubit 0: 
$|\psi\rangle' = cos(\theta_1/2)*|000\rangle+sin(\theta_1/2)*|100\rangle$

```plaintext
    ry(theta, qubit, label=None)
```

In [79]:
theta1 = 2*np.arcsin(psi[0]).real
qc.ry(theta1, 0)

2. Controlled rotation of qubit 1, ctrl being qubit 0:

```plaintext
    cry(theta, control_qubit, target_qubit, label=None, ctrl_state=None)
```

In [80]:
theta2 = 2*np.arcsin(psi[1]/np.sqrt(1-psi[0]**2)).real
qc.cry(theta2, 0, 1, ctrl_state=0)

3. Multi controlled rotation of qubit 2, ctrl being qubits 0 and 1

Visualization:
```plaintext
| Original (q0, q1) | After first Xs | Controlled gate acts? | After last Xs (restored) |
|--------------------|----------------|------------------------|---------------------------|
| 00                 | 11             | ✅ Yes                 | 00                        |
| 01                 | 10             | ❌ No                  | 01                        |
| 10                 | 01             | ❌ No                  | 10                        |
| 11                 | 00             | ❌ No                  | 11                        |

```

In [81]:
theta3 = 2*np.arcsin(psi[2]/np.sqrt(1-psi[0]**2-psi[1]**2)).real
qc.x(0)
qc.x(1)
qc.mcry(theta3, [0, 1], 2)
qc.x(0)
qc.x(1)

## 2. Unitary transformation in free space

$ U(t) = exp(-i*H*\delta t) $, with H being the Hamiltonian and $ \hbar \ = 1 $

## 3. Quantum Simulator

In [82]:
qc.measure([0, 1, 2], [0, 1, 2])

In [83]:
backend = QasmSimulator(method='statevector')
job = backend.run(qc, shots=np.pow(2,20))
result = job.result()

counts = result.get_counts()
print(counts)

{'010': 1048576}
